# Exploring Ebay Car Sales Data(Germany)

In this project will be analysed dataset of used cars from German Ebay website. The dataset was originally scraped and uploaded to Kaggle. 

The version of the dataset we are working with is a sample of 50,000 data points that was prepared by Dataquest including simulating a less-cleaned version of the data.

The data dictionary provided with data is as follows:

In [1]:
#Import libraries
import pandas as pd
import numpy as np

In [2]:
#Open and observe dataset
autos = pd.read_csv(r'autos.csv', encoding = 'Latin-1')
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [3]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The dataset consist of 20 columns and mostly (14 of 20) columns stored as string. Also several columns have null values.
The purpose is to clean data for more convinient work. Firstly, lets convert column names to more 'pythonic'.

### 1) Editing column names

In [4]:
column_names = autos.columns
column_names

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
# Let's convert the column names from camelcase to snakecase
new_names = {}
for column in column_names:
    new_column = ''
    for char in column:
        if char.isupper():
            new_column += '_' + char.lower()
        else:
            new_column += char
    new_names[column] = new_column
autos.rename(columns = new_names, inplace = True)

In [6]:
autos.head(3)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


### Initial Data Exploration and Cleaning
Now let's do some basic data exploration to determine what other cleaning tasks need to be done. 

Initially we will look for: 
- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis. 

- Examples of numeric data stored as text which can be cleaned and converted.

In [7]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Firstly, lets drop columns which have mostly one value:

In [8]:
#Columns to drop:
columns_to_drop = ['seller', 'offer_type', 'nr_of_pictures']
autos.drop(columns = columns_to_drop, inplace = True)
autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'year_of_registration', 'gearbox', 'power_p_s', 'model', 'odometer',
       'month_of_registration', 'fuel_type', 'brand', 'not_repaired_damage',
       'date_created', 'postal_code', 'last_seen'],
      dtype='object')

In [9]:
# Numeric data stored as text to numeric: price, odometer
autos['price'].value_counts()

$0         1421
$500        781
$1,500      734
$2,500      643
$1,000      639
           ... 
$4,994        1
$29,400       1
$51,990       1
$40,990       1
$3,120        1
Name: price, Length: 2357, dtype: int64

In [10]:
autos['odometer'].value_counts()

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64

In [11]:
autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',', '').astype(int)
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(int)
autos.rename(columns = {'odometer': 'odometer_km',
                       'price': 'price_$'}, inplace = True)

In [12]:
autos.head(3)

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37


In [13]:
autos.describe(include = 'all')

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen
count,50000,50000,5.000000e+04,50000,44905,50000.000000,47320,50000.000000,47242,50000.000000,50000.000000,45518,50000,40171,50000,50000.000000,50000
unique,48213,38754,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,2016-04-07 06:17:27
freq,3,78,NaN,25756,12859,NaN,36993,NaN,4024,NaN,NaN,30107,10687,35232,1946,NaN,8
mean,NaN,NaN,9.840044e+03,NaN,NaN,2005.073280,NaN,116.355920,NaN,125732.700000,5.723360,NaN,NaN,NaN,NaN,50813.627300,NaN
std,NaN,NaN,4.811044e+05,NaN,NaN,105.712813,NaN,209.216627,NaN,40042.211706,3.711984,NaN,NaN,NaN,NaN,25779.747957,NaN
min,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,1067.000000,NaN
25%,NaN,NaN,1.100000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,30451.000000,NaN
50%,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,49577.000000,NaN
75%,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,71540.000000,NaN


Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.

In [14]:
autos['price_$'].value_counts().sort_index(ascending = False)

99999999       1
27322222       1
12345678       3
11111111       2
10000000       1
            ... 
5              2
3              1
2              3
1            156
0           1421
Name: price_$, Length: 2357, dtype: int64

There are number of outliers which we can remove. For example, 1 421 cars listed with 0 price, also unrealisticly too high values like 99999$. 

I am going to remove cars with 0 price and anything more than 350 000$.

In [15]:
# Now autos df contains only reasonable values
autos = autos[(autos['price_$'] > 0) & (autos['price_$'] <= 350000)]
autos['price_$'].value_counts().sort_index(ascending = False)

350000      1
345000      1
299000      1
295000      1
265000      1
         ... 
8           1
5           2
3           1
2           3
1         156
Name: price_$, Length: 2346, dtype: int64

In [16]:
autos['odometer_km'].value_counts().sort_index(ascending = False)

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
40000       815
30000       780
20000       762
10000       253
5000        836
Name: odometer_km, dtype: int64

In [17]:
autos["price_$"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price_$, dtype: float64

In [18]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48565 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   date_crawled           48565 non-null  object
 1   name                   48565 non-null  object
 2   price_$                48565 non-null  int32 
 3   abtest                 48565 non-null  object
 4   vehicle_type           43979 non-null  object
 5   year_of_registration   48565 non-null  int64 
 6   gearbox                46222 non-null  object
 7   power_p_s              48565 non-null  int64 
 8   model                  46107 non-null  object
 9   odometer_km            48565 non-null  int32 
 10  month_of_registration  48565 non-null  int64 
 11  fuel_type              44535 non-null  object
 12  brand                  48565 non-null  object
 13  not_repaired_damage    39464 non-null  object
 14  date_created           48565 non-null  object
 15  postal_code        

### Exploring the date columns

Now lets check columns with data which assumed to be datetime type.

There are a number of columns with date information:

- date_crawled
- month_of_registration
- year_of_registration
- date_created
- last_seen

In [19]:
autos.loc[:5, ['date_crawled', 'month_of_registration', 'year_of_registration', 'date_created', 'last_seen']]

,date_crawled,month_of_registration,year_of_registration,date_created,last_seen
0,2016-03-26 17:47:46,3,2004,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,6,1997,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,7,2009,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,6,2007,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,7,2003,2016-04-01 00:00:00,2016-04-01 14:38:50
5,2016-03-21 13:47:45,4,2006,2016-03-21 00:00:00,2016-04-06 09:45:21


In [20]:
# For columns date_crawled, date_created, last_seen create function for date distribution
def date_distribution(name_column):
    column = autos[name_column]
    return column.str[:10].value_counts(normalize=True, dropna=False).sort_index()

In [21]:
date_distribution('date_crawled')

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

Looks like the site was crawled daily over roughly a one month period in March and April 2016. The distribution of listings crawled on each day is roughly uniform.

In [22]:
date_distribution('last_seen')

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [23]:
autos['year_of_registration'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: year_of_registration, dtype: float64

The year that the car was first registered will likely indicate the age of the car. 
Looking at this column, we note some odd values. 
The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

# Dealing with Incorrect Registration Year Data

Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:


In [24]:
autos['year_of_registration'].value_counts().sort_index()

1000    1
1001    1
1111    1
1800    2
1910    5
       ..
5911    1
6200    1
8888    1
9000    1
9999    3
Name: year_of_registration, Length: 95, dtype: int64

Lets remove unreasonable years of registration. The frist auto was invented in 1885, so our boundaries of years of registration column is 1900 to 2016.

In [25]:
autos = autos[(autos['year_of_registration'] >= 1900) & (autos['year_of_registration'] <= 2016)]
autos['year_of_registration'].value_counts(normalize=True).head(15)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
2008    0.047450
2009    0.044665
1997    0.041794
2011    0.034768
2010    0.034040
Name: year_of_registration, dtype: float64

It appears that most of the vehicles were first registered in the past 20 years.

In [26]:
brands = autos['brand'].unique()

In [27]:
autos['brand'].value_counts()

volkswagen        9862
bmw               5137
opel              5022
mercedes_benz     4503
audi              4041
ford              3263
renault           2201
peugeot           1393
fiat              1197
seat               853
skoda              766
nissan             713
mazda              709
smart              661
citroen            654
toyota             593
hyundai            468
sonstige_autos     458
volvo              427
mini               409
mitsubishi         384
honda              366
kia                330
alfa_romeo         310
porsche            286
suzuki             277
chevrolet          266
chrysler           164
dacia              123
daihatsu           117
jeep               106
subaru             100
land_rover          98
saab                77
jaguar              73
daewoo              70
trabant             65
rover               62
lancia              50
lada                27
Name: brand, dtype: int64

In [28]:
autos['price_to_km'] = autos['price_$'] / autos['odometer_km']
autos.head()

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen,price_to_km
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54,0.033333
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08,0.056667
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37,0.128429
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28,0.062143
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50,0.009000


In [29]:
top_price_to_km = {}
for brand in brands:
    row_selected = autos[autos['brand'] == brand]
    mean_price_to_km = row_selected['price_to_km'].mean()
    top_price_to_km[brand] = mean_price_to_km

In [30]:
import operator

In [31]:
sorted_d = dict(sorted(top_price_to_km.items(), key=operator.itemgetter(1),reverse=True))
sorted_d

{'porsche': 1.8709126440503943,
 'sonstige_autos': 0.5679373536424758,
 'land_rover': 0.5462690090702947,
 'jaguar': 0.2732215398999782,
 'mini': 0.2634142910699732,
 'jeep': 0.2390813966756514,
 'audi': 0.17354781584198473,
 'dacia': 0.16960468666924763,
 'chevrolet': 0.15822726062179257,
 'skoda': 0.13832622537195904,
 'mercedes_benz': 0.13729314837903478,
 'bmw': 0.1345454733400076,
 'hyundai': 0.1164135084622168,
 'subaru': 0.11222695230158729,
 'kia': 0.11174816895141895,
 'toyota': 0.09776267896624641,
 'lada': 0.09414157172251615,
 'alfa_romeo': 0.0932228632360471,
 'volkswagen': 0.09114396998902313,
 'trabant': 0.08987870634920635,
 'mazda': 0.08815440216490923,
 'seat': 0.08773534840618546,
 'ford': 0.08433656602162777,
 'nissan': 0.08416590868006858,
 'suzuki': 0.07919699776517104,
 'honda': 0.07637797113799984,
 'citroen': 0.06894431263045482,
 'fiat': 0.06764621300606012,
 'smart': 0.06304248232596114,
 'mitsubishi': 0.06255475905257936,
 'opel': 0.05591560532229617,
 'volv

We can conclude that porsche is less exposed to depreciation for every km.

In [32]:
# Lets create a list of most popular autos in Germany
brand_counts = autos['brand'].value_counts(normalize = True)
common_brands = brand_counts[brand_counts > 0.05].index

In [33]:
# For each brand lets calculate mean price
common_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

In [34]:
brand_mean_price = {}
for brand in common_brands:
    row_selected = autos[autos['brand'] == brand]
    mean_price =  row_selected['price_$'].mean()
    brand_mean_price[brand] = int(mean_price)
brand_mean_price

{'volkswagen': 5402,
 'bmw': 8332,
 'opel': 2975,
 'mercedes_benz': 8628,
 'audi': 9336,
 'ford': 3749}

Of the top 5 brands, there is a distinct price gap:

- Audi, BMW and Mercedes Benz are more expensive
- Ford and Opel are less expensive
- Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option.

In [35]:
sorted_brand_mean_price = dict(sorted(brand_mean_price.items(), key=operator.itemgetter(1),reverse=True))

In [36]:
mean_prices = pd.Series(sorted_brand_mean_price).sort_values(ascending = False)

In [37]:
brand_mean_milleage = {}
for brand in common_brands:
    row_selected = autos[autos['brand'] == brand]
    mean_milleage = row_selected['odometer_km'].mean()
    brand_mean_milleage[brand] = int(mean_milleage)
mean_milleage = pd.Series(brand_mean_milleage).sort_values(ascending = False)

In [38]:
df = pd.DataFrame(mean_milleage, columns = ['mean_milleage'])

In [39]:
df['price'] = mean_prices

In [40]:
df

,mean_milleage,price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.

In [41]:
autos.head()

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen,price_to_km
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54,0.033333
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08,0.056667
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37,0.128429
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28,0.062143
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50,0.009000


In [42]:
autos['vehicle_type'].value_counts()

limousine     12598
kleinwagen    10585
kombi          8930
bus            4031
cabrio         3016
coupe          2462
suv            1965
andere          390
Name: vehicle_type, dtype: int64

In the next step for some columns(vehicle_type) swap german words for english words. By using map function.

In [43]:
autos.head()

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen,price_to_km
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54,0.033333
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08,0.056667
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37,0.128429
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28,0.062143
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50,0.009000


In [44]:
vehicle_type_to_english = {'limousine': 'limousine',
                           'bus': 'bus',
                           'kleinwagen': 'small_car',
                           'kombi': 'estate_car',
                           'cabrio': 'convertible',
                           'suv': 'off_road',
                           'andere': 'other',
                           'coupe': 'coupe'}
autos['vehicle_type'] = autos['vehicle_type'].map(vehicle_type_to_english)

In [45]:
autos['vehicle_type'].value_counts()

limousine      12598
small_car      10585
estate_car      8930
bus             4031
convertible     3016
coupe           2462
off_road        1965
other            390
Name: vehicle_type, dtype: int64

In [46]:
autos.head()

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen,price_to_km
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54,0.033333
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08,0.056667
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37,0.128429
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,small_car,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28,0.062143
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,estate_car,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50,0.009000


In [47]:
# Convert the dates to be uniform numeric data, so "2016-03-21" becomes the integer 20160321.
date_columns = ['date_crawled', 'date_created', 'last_seen']

In [48]:
for column in date_columns:
    autos[column] = autos[column].str[:10].str.replace('-', '').astype(int)

In [49]:
autos.head()

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen,price_to_km
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,20160326,79588,20160406,0.033333
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,20160404,71034,20160406,0.056667
2,20160326,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,20160326,35394,20160406,0.128429
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,small_car,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,20160312,33729,20160315,0.062143
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,estate_car,2003,manuell,0,focus,150000,7,benzin,ford,nein,20160401,39218,20160401,0.009000


In the next step lets find the most common brand/model combination.

In [50]:
unique_model = autos['name'].unique()

In [51]:
autos['name'].value_counts()

BMW_316i                                        75
Volkswagen_Golf_1.4                             75
Ford_Fiesta                                     74
BMW_318i                                        72
Volkswagen_Polo                                 72
                                                ..
Ford_Escort_Cabrio_1.6_16V_Karmann               1
Renault_Modus_1.2_16V_Avantage                   1
RENAULT_MEGANE_GT_LINE_MIT_GARANTIE_BIS_2018     1
BMW_Z4_sDrive23i_Aut._Vollausstattung            1
Golf_1_9_TDI__fahrbereit                         1
Name: name, Length: 35812, dtype: int64

##### The most most common combination is BMW: BMW_316i and Volkswage: Volkswagen_Golf_1.4

Lets see if average prices follows any patterns based on the milage.

In [52]:
autos['odometer_km'].value_counts().sort_index(ascending = False)

150000    30085
125000     4857
100000     2058
90000      1673
80000      1375
70000      1187
60000      1128
50000       993
40000       797
30000       760
20000       742
10000       241
5000        785
Name: odometer_km, dtype: int64

Lets split odometer_km column into three groups: Less or equal 50000km, Less or equal 100000, More than 100000

In [53]:
autos.loc[autos['odometer_km'] <= 50000, 'new_column'] = 'Less 50000km'

In [54]:
autos.loc[(autos['odometer_km'] > 50000) & (autos['odometer_km'] <= 100000), 'new_column'] = 'more than 50000 less 100000km'

In [55]:
autos.loc[autos['odometer_km'] > 100000, 'new_column'] = 'More 100000km'

In [56]:
autos.head()

,date_crawled,name,price_$,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer_km,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,postal_code,last_seen,price_to_km,new_column
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,20160326,79588,20160406,0.033333,More 100000km
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,20160404,71034,20160406,0.056667,More 100000km
2,20160326,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,20160326,35394,20160406,0.128429,more than 50000 less 100000km
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,small_car,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,20160312,33729,20160315,0.062143,more than 50000 less 100000km
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,estate_car,2003,manuell,0,focus,150000,7,benzin,ford,nein,20160401,39218,20160401,0.009000,More 100000km


In [57]:
unique_new_column = autos['new_column'].unique()
unique_new_column

array(['More 100000km', 'more than 50000 less 100000km', 'Less 50000km'],
      dtype=object)

In [60]:
price_by_km = {}
for km in unique_new_column:
    selected_row = autos[autos['new_column'] == km]
    mean_price = selected_row['price_$'].mean()
    price_by_km[km] = int(mean_price)

In [61]:
price_by_km

{'More 100000km': 4107,
 'more than 50000 less 100000km': 9595,
 'Less 50000km': 14890}

##### As we have expected cars with a lower mileage more expensive

In [68]:
autos['not_repaired_damage'].value_counts(dropna = False)

nein    33834
NaN      8307
ja       4540
Name: not_repaired_damage, dtype: int64

In [66]:
unique = autos['not_repaired_damage'].unique()
unique

array(['nein', nan, 'ja'], dtype=object)

In [73]:
price_damaged_or_not = {}
for x in unique:
    row_selected = autos[autos['not_repaired_damage'] == x]
    mean_price = row_selected['price_$'].mean()
    price_damaged_or_not[x] = mean_price
price_damaged_or_not

{'nein': 7164.033102796004, nan: nan, 'ja': 2241.146035242291}

Cars with damage are cheaper than non-damage on 5000 dollars on average.